# Langchain to Bedrock Migration

In [ ]:
%pip install langchain langchain-aws boto3 --upgrade

## Step 0: Prerequisites and setup

In [ ]:
import time
import boto3
import logging
import ipywidgets as widgets
import uuid

from agent import create_agent_role, create_lambda_role
from agent import create_dynamodb, create_lambda, invoke_agent_helper

#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
# region, account_id
agent_name = 'my-agent-'+region+account_id + str(time.time()).replace('.','-')
agent_name

## Step 1: Create and edit a local `lambda_function.py` 

In [ ]:
%%writefile lambda_function.py
import json
import random
import uuid
        

def lambda_handler(event, context):

    print(event)
    
    ## Langchain wraps request and response in 'body', but Bedrock agents does not
    if 'body' in event:
        event = json.loads(event['body'])
    agent = event['agent']
    actionGroup = event['actionGroup']
    function = event['function']
    # parameters = event.get('parameters', [])
    responseBody =  {
        "TEXT": {
            "body": "Error, no function was called"
        }
    }


    # random number generator
    if function == 'get_random':
        out = str(random.random())
    
    # UUID4 generator
    elif function == 'get_uuid':
        out = str(uuid.uuid4())
        
    else:
        out = f"output: Function called is invalid: {function}"
        
    responseBody =  {
            'TEXT': {
                "body": out
            }
        }
    
    action_response = {
        'actionGroup': actionGroup,
        'function': function,
        'functionResponse': {
            'responseBody': responseBody
        }

    }
    
    ## Langchain wraps request and response in 'body', but Bedrock agents does not
    
    if isinstance( event['agent'] , dict):
        function_response = {'response': action_response, 'messageVersion': event['messageVersion']}
    else:
        function_response = {'body': {'response': action_response, 'messageVersion': event['messageVersion']}}
        
    
    print("Response: {}".format(function_response))

    return function_response


## Step 2: Create Lambda function

In [ ]:
lambda_iam_role = create_lambda_role(agent_name, dynamodb_table_name='') # Do not create DynamoDB table. If your Lambda requires DynamoDB, provide a table name
lambda_function_name = f'{agent_name}-lambda'
lambda_function = create_lambda(lambda_function_name, lambda_iam_role, path_to_lambda = 'lambda_function.py')

## Step 3: Test model invoke and Lambda invoke

#### Test Model invoke separately

In [ ]:
from langchain_aws import ChatBedrock, BedrockLLM

In [ ]:
import boto3
client = boto3.client("bedrock")
model_list = [s['modelId'] for s in client.list_foundation_models()['modelSummaries']]

In [ ]:
import ipywidgets

agent_llm_selector = ipywidgets.Dropdown(
    options=[(s,s) for s in model_list]
)

agent_llm_selector

In [ ]:
llm = ChatBedrock(model_id=agent_llm_selector.value)

In [ ]:
llm.invoke(input="Hello, ")

#### Test Lambda invoke separately:

We will be mocking the test event that is required when using Amazon Bedrock Agents

In [ ]:
import json 
lambda_client = boto3.client('lambda')

test_event = {
  "agent": "1234",
  "actionGroup": "5678",
  "messageVersion":"9101112",
  "function": "get_uuid",
}

res = lambda_client.invoke(
            FunctionName=lambda_function_name,
            InvocationType="RequestResponse",
            Payload=json.dumps(test_event),
        )
            


In [ ]:
json.dumps(test_event)

In [ ]:
payload_stream = res["Payload"]
payload_string = payload_stream.read().decode("utf-8")
answer = json.loads(payload_string)#["body"]
answer

## Step 4: Test on Langchain

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools

tools = load_tools(
    ["awslambda"],
    awslambda_tool_name="get_random_or_uuid",
    awslambda_tool_description="""Returns a random number or a UUID based on choice of function. Pass in json that has the following keys --- 1. 'agent':(random id); 2. 'actionGroup':(random id); 3. 'function':choose between 'get_random' or 'get_uuid'; 4. 'messageVersion': (random version) """,
    function_name=lambda_function['FunctionName']
)

agent = initialize_agent(
    tools,
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True
)

agent.run("Generate a random number, use that as a probability of a coin flip (<0.5 is heads). If the coin is heads, generate a UUID.")

## Step 5: Deploy to Amazon Bedrock Agents

In [ ]:
agent_name

In [ ]:
suffix = f"{region}-{account_id}"

agent_bedrock_allow_policy_name = f"{agent_name}-ba"
agent_role_name = f'role_{agent_name}'[:64]

agent_description = "Uses tools like random number generation and UUID generation to answer questions"

agent_instruction = tools[0].description

agent_action_group_description = "Action for " + agent_description

agent_action_group_name = suffix+"ActionGroup"

#### 5a. Create agent role

In [ ]:
agent_role = create_agent_role(agent_name, agent_llm_selector.value)

In [ ]:
agent_role

#### 5b. Create agent

In [ ]:
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_llm_selector.value,
    instruction=agent_instruction,
)

agent_id = response['agent']['agentId']
print("The agent id is:",agent_id)

#### 5c. Create agent action group

In [ ]:
agent_functions = [
    {
        'name': tools[0].name,
        'description': tools[0].description,
        'parameters': {}
    },]

In [ ]:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)

In [ ]:
agent_action_group_response['agentActionGroup']

#### 5d. Allow Bedrock to call Lambda functions

In [ ]:
# Create allow to invoke permission on lambda
lambda_client = boto3.client('lambda')
try:
    response = lambda_client.add_permission(
        FunctionName=lambda_function_name,
        StatementId=f'allow_bedrock_{agent_id}',
        Action='lambda:InvokeFunction',
        Principal='bedrock.amazonaws.com',
        SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
    )
    print(response)
except Exception as e:
    print(e)

#### 5e. Prepare the agent

In [ ]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)

#### 5f. Invoke the migrated agent

In [ ]:
%%time
session_id:str = str(uuid.uuid1())
alias_id = 'TSTALIASID'
query = "Generate a random number, use that as a probability of a coin flip (<0.5 is heads). If the coin is heads, generate a UUID. Provide the final answer in JSON as {'coin_state':'heads','uuid':'<insert UUID if generated otherwise use " ">'}"
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

## Step 6: Cleanup!

In [ ]:
from agent import delete_agent_roles_and_policies, clean_up_resources

try:
    clean_up_resources(
    '', lambda_function, lambda_function_name, agent_action_group_response, agent_functions, 
    agent_id, '', alias_id)
    
    delete_agent_roles_and_policies(agent_name, kb_policy_name)
except Exception as e:
    print(e)
